In [1]:
import cv2 as cv
import poly_point_isect as bot
import math
import matplotlib.pyplot as plt
import numpy as np
import sys

## Image Acquisition

In [2]:
img = cv.imread(cv.samples.findFile("example.png"))
if img is None:
    sys.exit("Could not read the image.")

cv.imshow("Display window", img)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example.png", img)

## Image Pre-Processing

### Grayscale Conversion

In [3]:
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

cv.imshow("Display window", gray)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example.png", gray)

### Noise Reduction

In [4]:
blur_gray = cv.GaussianBlur(gray, (5, 5), 0)

cv.imshow("Display window", blur_gray)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example.png", blur_gray)

### Binarization

In [5]:
ret, binary_blur_gray = cv.threshold(blur_gray, 127, 255, cv.THRESH_BINARY)

cv.imshow("Display window", binary_blur_gray)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example.png", binary_blur_gray)

### Edge and Line Detection

In [6]:
binary_blur_rgb = cv.cvtColor(binary_blur_gray, cv.COLOR_GRAY2RGB)

line_img = np.zeros((binary_blur_rgb.shape[0], binary_blur_rgb.shape[1], 3), dtype=np.uint8)
line_img[:] = (255, 255, 255)

# edge detection
lower_threshold = 50  # lower threshold value in Hysteresis Thresholding
upper_threshold = 150  # upper threshold value in Hysteresis Thresholding
aperture_size = 7  # aperture size of the Sobel filter
edges = cv.Canny(binary_blur_rgb, lower_threshold, upper_threshold, aperture_size)

# line detection
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 20  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 30  # minimum number of pixels making up a line
max_line_gap = 10  # maximum gap in pixels between connectable line segments
lines = cv.HoughLinesP(edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
new_lines = [[[x1, y1], [x2, y2]] for [[x1, y1, x2, y2]] in lines]

# Draw the lines
for line in new_lines:
    x1, y1 = line[0]
    x2, y2 = line[1]
    cv.line(line_img, (x1, y1), (x2, y2), (255, 0, 0), 5)

new_img = cv.addWeighted(binary_blur_rgb, 0.8, line_img, 1, 0)

cv.imshow("Display window", new_img)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example_lines_blue.png", new_img)

# Intersection Detection

In [ ]:
intersection_img = np.zeros((binary_blur_rgb.shape[0], binary_blur_rgb.shape[1], 3), dtype=np.uint8)
intersection_img[:] = (255, 255, 255)

# Intersections computation
def lines_similar(line1, line2, slope_threshold):
    # Calculate slopes of the lines
    slope1 = (line1[1][1] - line1[0][1]) / (line1[1][0] - line1[0][0]) if line1[1][0] != line1[0][0] else 1000000
    slope2 = (line2[1][1] - line2[0][1]) / (line2[1][0] - line2[0][0]) if line2[1][0] != line2[0][0] else 1000000

    # Compare slopes
    return abs(slope1 - slope2) < slope_threshold

def line_intersection(line1, line2):
    # Unpack lines
    x1, y1, x2, y2 = line1[0][0], line1[0][1], line1[1][0], line1[1][1]
    x3, y3, x4, y4 = line2[0][0], line2[0][1], line2[1][0], line2[1][1]

    # Compute determinants
    det1_and_2 = det(x1, y1, x2, y2)
    det3_and_4 = det(x3, y3, x4, y4)

    # Compute x and y coordinates of intersection point
    x = (det(det1_and_2, x1 - x2, det3_and_4, x3 - x4) / det(x1 - x2, y1 - y2, x3 - x4, y3 - y4))
    y = (det(det1_and_2, y1 - y2, det3_and_4, y3 - y4) / det(x1 - x2, y1 - y2, x3 - x4, y3 - y4))

    # Check if intersection point falls within the domain and range of both lines
    if min(x1, x2) <= x <= max(x1, x2) and min(y1, y2) <= y <= max(y1, y2) and min(x3, x4) <= x <= max(x3, x4) and min(y3, y4) <= y <= max(y3, y4):
        return x, y
    else:
        return None


def det(a, b, c, d):
    return a * d - b * c

intersections = []
for i in range(len(new_lines)):
    for j in range(i+1, len(new_lines)):
        if not lines_similar(new_lines[i], new_lines[j], 0.5):  # 0.1 as slope threshold
            intersection = line_intersection(new_lines[i], new_lines[j])
            if intersection:  # if lines intersect
                intersections.append(intersection)              

# Intersections merging
def distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2) 

merged_intersections = []
for point in intersections:  
    nearby_points = [p for p in intersections if distance(p, point) < 40]  # 4 cm is 40 pixels
    merged_point = [sum(p[i] for p in nearby_points) / len(nearby_points) for i in range(2)]
    merged_intersections.append(merged_point)
merged_intersections = list(set(tuple(p) for p in merged_intersections))

# Draw the intersections
for intersection in merged_intersections:
    cv.circle(new_img, tuple(map(int, intersection)), 5, (0, 255, 0), -1)  # -1 to fill the circle

cv.imshow("Display window", new_img)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("example_lines_blue.png", new_img)

## Post-Processing